# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [54]:
#load in data base
db = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)

In [55]:
db.head(5)

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,celia,08032017_Baby,Baby,2017-08-03,0.0,0,1.0,1.0,1000.0,1000.0,3.0,0.0,0.0,0.0,10.0,1.0,50.0,50.0
1,celia,08042017_Baby,Baby,2017-08-04,1.0,0,0.9,0.9,1000.0,1000.0,19.0,0.0,15.0,0.0,15.0,1.0,60.0,60.0
2,celia,08072017_Baby,Baby,2017-08-07,1.0,0,0.9,0.9,1000.0,1000.0,60.0,0.0,51.0,0.0,15.0,1.0,50.0,50.0
3,celia,08082017_Baby,Baby,2017-08-08,1.0,0,0.9,0.9,1000.0,1000.0,76.0,0.0,71.0,0.0,10.0,1.0,50.0,50.0
4,celia,08092017_Baby,Baby,2017-08-09,1.0,0,0.9,0.9,1000.0,1000.0,108.0,0.0,96.0,0.0,5.0,1.0,45.0,45.0


### Query all 80-20 sessions where performance > 0.7 and block structure was 50

In [56]:
r = db[(db['Markov']==1)]
#r = db[((db['Left Reward Prob'] == 0.8) |  (db['Right Reward Prob'] == 0.8))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
#r = r[r['Mouse ID'] == 'Mustard']
session_names = r['Session ID'].values

In [57]:
r.shape

(60, 18)

## load in csv files (from running exportTrials.m)

In [58]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [50]:
len(trial_df)

55

## convert into 1 feature matrix

In [ ]:
trial_df

In [59]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [ ]:
trials = df.copy()
n_indi=10
curr_trial_duration=True
imaging=False

n_trials = trials.shape[0] #number of trials in this session
    
#minimum number of columns
num_cols = 2 + 3+ 4*n_indi+ 1 + 4
#2 streak cols, 3 trial & block & reward trial col, 4 cols for each past trial, 
#]1 for current trial + 4 decision/switch/higher p port/Reward
    
#add columns as neccesary
if curr_trial_duration is True:
    num_cols += 1
if imaging is True:
    num_cols += 2
    
feature_matrix = np.zeros((n_trials-n_indi,num_cols))
    
block_starts = np.zeros(trials.shape[0])
block_starts[1:] = np.diff(trials['Right Reward Prob'].values) != 0
    
reward_block_num = 0    
reward_block_nums = np.zeros(n_trials)
for i in range(n_trials):
    if block_starts[i]:
        reward_block_num = 0
    else:
        reward_block_num += trials.iloc[i]['Reward Given']
    
    reward_block_nums[i] = reward_block_num

for j,i in enumerate(np.arange(n_indi,n_trials)):
    k = 0 #column indexer for the feature matrix
    #extract the 'n_summary' trials we need to consider. Assume that n_summary > n_indi
    past_trials = trials.iloc[i-n_indi:i]
        
    '''
    Trial Number
    '''
    feature_matrix[j,k] = i+1
    k+=1 
    # will be added after all at once (since can be calculated for every trial with single line)
        
    '''
    Block Trial Number
    '''
    if (j == 0): #first block number will be the sum
        if (np.sum(block_starts[:n_indi]) == 0):
        #then we are still in the first block, and we simply are n_indi trials in
            feature_matrix[j,k] = n_indi+1
        else:
            feature_matrix[j,k] = n_indi - np.where(block_starts[:n_indi]==True)[0][0] + 1
            # i.e. 10 - 5 + 1 = 6 or 10 - 8 + 1 = 3. it works. 
    elif block_starts[i]: #if block_starts[j] == True, start counting from 0
        feature_matrix[j,k] = 0 
    else:
        feature_matrix[j,k] = feature_matrix[j-1,k] + 1
            
    k+=1
        
    '''
    Block Reward Number
    '''
    feature_matrix[j,k] = reward_block_nums[i-1]
    k+=1
            

In [ ]:
np.where(block_starts[:n_indi]==True)[0][0]

In [ ]:
block_starts

In [ ]:
master_matrix.head(5)

In [9]:
master_matrix.shape

(19031, 53)

In [60]:
master_matrix['Higher p port'].mean()

0.8492503354530074

## Save combined feature matrix  

In [61]:
master_matrix.to_csv(os.path.join(save_dir,'markov_master_df.csv'),index=False)